**Imports**

In [9]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import numpy as np
from qiskit_ibm_runtime.fake_provider import FakeManilaV2


**Initilize values**

In [10]:
numpy_pi = np.pi
num_qubits = 4
simulator = AerSimulator()
backend = FakeManilaV2()
qc = QuantumCircuit(num_qubits, num_qubits) # qubits, classical bits


Gates

In [58]:
#redundant h and cNOT gates on the same qubit(s)
qc.h(0)
qc.h(0)
qc.cx(0, 1)
qc.cx(0, 1)

# just a full rotation(2pi) around the x-axis and a (3/2)pi rotation in the z-axis
# completely redundant.
qc.rx((0.5)*numpy_pi,3)

qc.rz((0.5)*numpy_pi,2)
qc.rx((0.5)*numpy_pi,3)
qc.rz(numpy_pi,2)

qc.rx((0.5)*numpy_pi,3)
qc.rx((0.5)*numpy_pi,3)

qc.cx(0,3) # CNOT is non-adjacent

qc.cx(1,2)


qc.measure(range(num_qubits), range(num_qubits))

Print

In [59]:
traspiled_qc_level_00 = transpile(qc,simulator,optimization_level=0)
traspiled_qc_level_01 = transpile(qc,simulator,optimization_level=1)
print("Original Circuit depth:", qc.depth())
print(qc)

print("Transpiled circuit w optimization 00:", traspiled_qc_level_00.depth())
print(traspiled_qc_level_00)

print("Transpiled circuit w optimization 01:", traspiled_qc_level_01.depth())
print(traspiled_qc_level_01)

traspiled_qc_level_02 = transpile(qc,simulator,optimization_level=2)
print("Transpiled circuit w optimization 02:", traspiled_qc_level_02.depth())
print(traspiled_qc_level_02)

traspiled_qc_level_03 = transpile(qc,simulator,optimization_level=3)
print("Transpiled circuit w optimization 03:", traspiled_qc_level_03.depth())
print(traspiled_qc_level_03)


Original Circuit depth: 6
        ┌───┐      ┌───┐                                   ┌─┐      
q_0: ───┤ H ├──────┤ H ├────────■──────────■───────■───────┤M├──────
        └───┘      └───┘      ┌─┴─┐      ┌─┴─┐     │       └╥┘┌─┐   
q_1: ─────────────────────────┤ X ├──────┤ X ├─────┼────■───╫─┤M├───
     ┌─────────┐ ┌───────┐    └───┘      └───┘     │  ┌─┴─┐ ║ └╥┘┌─┐
q_2: ┤ Rz(π/2) ├─┤ Rz(π) ├─────────────────────────┼──┤ X ├─╫──╫─┤M├
     ├─────────┤┌┴───────┴┐┌─────────┐┌─────────┐┌─┴─┐└┬─┬┘ ║  ║ └╥┘
q_3: ┤ Rx(π/2) ├┤ Rx(π/2) ├┤ Rx(π/2) ├┤ Rx(π/2) ├┤ X ├─┤M├──╫──╫──╫─
     └─────────┘└─────────┘└─────────┘└─────────┘└───┘ └╥┘  ║  ║  ║ 
c: 4/═══════════════════════════════════════════════════╩═══╩══╩══╩═
                                                        3   0  1  2 
Transpiled circuit w optimization 00: 6
        ┌───┐      ┌───┐                                   ┌─┐         
q_0: ───┤ H ├──────┤ H ├────────■──────────■────────────■──┤M├─────────
        └───┘      └───┘      ┌

| Optimization Level | Description |
|--------------------|-------------|
| **0** | **No optimization:** typically used for hardware characterization<br><br>• Basic translation<br>• Layout/Routing: `TrivialLayout`, where it selects the same physical qubit numbers as virtual and inserts SWAPs to make it work (using `SabreSwap`) |
| **1** | **Light optimization:**<br><br>• Layout/Routing: First attempts `TrivialLayout`. If additional SWAPs are needed, a layout with the minimum number of SWAPs is searched using `SabreSwap`. Then `VF2LayoutPostLayout` tries to choose better qubits.<br>• `InverseCancellation` (removes pairs of gate inverses)<br>• 1-qubit gate optimization |
| **2** | **Med optimization:**<br><br>• Layout/Routing: Optimization level 1 (without trivial) + heuristic optimized with greater search depth and trials of optimization function. Because TrivialLayout is not used, there is no attempt to use the same physical and virtual qubit numbers.<br>• `CommutativeCancellation` (Removes gates that commute AND cancel, even if they’re not adjacent)|
| **3** | **High optimization:**<br><br>• Optimization level 2, plus heuristic layout/routing optimized with greater search effort/trials<br>• Resynthesis of two-qubit blocks using Cartan's KAK decomposition<br>• Unitarity-breaking passes:<br>&nbsp;&nbsp;&nbsp;&nbsp;• `OptimizeSwapBeforeMeasure`: moves measurements earlier in the circuit to avoid introducing SWAPs<br>&nbsp;&nbsp;&nbsp;&nbsp;• `RemoveDiagonalGatesBeforeMeasure`: removes diagonal gates before measurement since they do not affect measurement results |
# 📘 Analysis of Optimization level 0 and 1


- **Optimization Level 0:**  
  Uses only essential transformations; preserves hardware logic with minimal changes.

- **Optimization Level 1:**  
